In [20]:

import pandas as pd
import os
Drug = pd.read_csv(os.path.join("webmd.csv"))
Drug = pd.read_csv("webmd.csv")
Drug.head()




,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1


In [21]:
 ## Split the Drug data into suitable X and y variables
##Drug[""] = Drug[""] + Drug[""] + 1
X = Drug[["EaseofUse", "Effectiveness","DrugId", "Sex"]]
y = Drug[["Satisfaction"]]
X.head()

,EaseofUse,Effectiveness,DrugId,Sex
0,5,5,146724,Male
1,5,5,146724,Female
2,2,3,144731,Female
3,2,2,144731,Female
4,1,1,144731,Male


In [22]:
##zeigt wie viele Nan vorhanden sind
len(X[X.isna().any(axis=1)])

from sklearn.impute import SimpleImputer
import numpy as np
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
si.fit(X)
X_ = si.transform(X)
X = pd.DataFrame(X_, columns=X.columns)
len(X[X.isna().any(axis=1)])

0

In [23]:
from sklearn import preprocessing
oe = preprocessing.OrdinalEncoder().fit(X[["Sex"]])
X[["Sex"]] = oe.transform(X[["Sex"]])

In [24]:
from sklearn import preprocessing
standard_scaler = preprocessing.StandardScaler().fit(X[["EaseofUse", "Effectiveness"]])

def scale(dataframe, columns, scaler):
    scaled = scaler.transform(dataframe[columns])
    dataframe[columns] = pd.DataFrame(scaled, columns=columns)
    return dataframe

X = scale(X, ["EaseofUse", "Effectiveness"], standard_scaler)
X.head()

,EaseofUse,Effectiveness,DrugId,Sex
0,0.743845,1.008228,146724,2.0
1,0.743845,1.008228,146724,1.0
2,-1.532833,-0.365437,144731,1.0
3,-1.532833,-1.052269,144731,1.0
4,-2.291725,-1.739102,144731,2.0


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size = 0.2, random_state = 42
)

len(X_train), len(X_test)

(290244, 72562)

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

algorithms = {
    "Decision Tree" : DecisionTreeClassifier(max_depth=5),
    "Nearest Neighbors" : KNeighborsClassifier(3),
    "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "Naive Bayes" : GaussianNB(),
    "LDA" : LinearDiscriminantAnalysis(),
}
for name, algorithm in algorithms.items():
    algorithm.fit(X_train, y_train.values.ravel())
    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))

Decision Tree 0.67
Nearest Neighbors 0.59
Random Forest 0.62
Naive Bayes 0.53
LDA 0.61


Als erstes habe ich jeden Algorithmus von der Aufgabe 1651 Sklearn ausprobiert und es stellte sich heraus, dass sich der Decision Tree am besten geeignet ist.Der Decision Tree geht nach einem ja nein Schema vor passt diese Aussage zur anderen und bestimmt so das Ergebnis. 3 Algorithmen sind oben nicht ersichtlich wegen der Rechenzeit. Die Rechenzeit betrug über 15min.

In [27]:
##Modell Speichern
import joblib
joblib.dump(algorithms["Decision Tree"], 'Drug_Decision_Tree.joblib')
best_model = joblib.load('Drug_Decision_Tree.joblib')
best_model

DecisionTreeClassifier(max_depth=5)

In [39]:

Test1 = pd.DataFrame([[5,5,146724,1]], columns=X_train.columns)
Test1 = scale(Test1, ["EaseofUse", "Effectiveness"], standard_scaler)
Test1.head()

Test2 = pd.DataFrame([[5,4,146724,0]], columns=X_train.columns)
Test2 = scale(Test2, ["EaseofUse", "Effectiveness"], standard_scaler)
Test2.head()
Ausgabe = pd.concat([Test1, Test2])
Ausgabe.head()

,EaseofUse,Effectiveness,DrugId,Sex
0,0.743845,1.008228,146724,1
0,0.743845,0.321396,146724,0
